In [1]:
from pandas import concat
from pandas import read_csv
from keras.layers import LSTM
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import datasets
import scipy.stats as stats
from keras.models import Sequential, Model
from keras.layers import Input
from keras.layers import LSTM, Dense, Dropout, Bidirectional, Flatten
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasRegressor
import tensorflow as tf
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from sklearn.model_selection import cross_val_score, StratifiedKFold
from evolutionary_search import EvolutionaryAlgorithmSearchCV
from evolutionary_search import maximize
import random

In [1]:
!pip install sklearn-deap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 139 kB 4.0 MB/s 


In [3]:
# Set random seed for reproducibility
tf.random.set_seed(1234)

# Bank BCA

In [2]:
# load data
reframed = read_csv("stock_dataset/normalisasi_bc.csv")

In [3]:
# pembagian data pelatihan dan pengujian
values = reframed.values

train_size = 0.8
valid_size=0.1

train_index = int(len(values)*train_size)

train = values[0:train_index]
rem = values[train_index:]

valid_index = int(len(values)*valid_size)

valid = values[train_index:train_index+valid_index]
test = values[train_index+valid_index:]

print(train.shape, valid.shape)

(1396, 8) (174, 8)


In [4]:
# membuat dataset
train_X, train_y = train[:, :-1], train[:, -1]
valid_X, valid_y = valid[:, :-1], valid[:, -1]

# reshape masukan menjadi 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
valid_X = valid_X.reshape((valid_X.shape[0], 1, valid_X.shape[1]))
print(train_X.shape, train_y.shape, valid_X.shape, valid_y.shape)

(1396, 1, 7) (1396,) (174, 1, 7) (174,)


In [5]:
def biLSTM_bc(units, units_1, batch_size, epochs, activation, lr):
    model = Sequential()
    # bidirectinal layer
    model.add(Bidirectional(LSTM(units, return_sequences=True), 
                            input_shape=(train_X.shape[1], train_X.shape[2])))
    # fully connected layer
    model.add(Flatten())
    model.add(Dense(units_1, activation=activation))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    opt = tf.keras.optimizers.Adam(lr)
    model.compile(optimizer = opt, loss= 'mse', metrics=[tf.keras.metrics.RootMeanSquaredError()])
    history = model.fit(train_X, train_y, 
                        batch_size=batch_size, 
                        epochs=epochs, validation_data=(valid_X, valid_y), 
                        verbose=0, shuffle=False)
    return model

In [6]:
rf_params = {
    'activation': ['relu','tanh', 'sigmoid'],
    'batch_size': [250, 350, 450],
    'units': range(30, 160),
    'units_1': range(40, 120),
    'epochs': [500, 1000],
    'lr':[0.0001, 0.0005]
}
clf = KerasRegressor(build_fn=biLSTM_bc, verbose=0)

random.seed(123)

# Set the hyperparameters of GA    
ga1 = EvolutionaryAlgorithmSearchCV(estimator=clf,
                                   params=rf_params,
                                   scoring="neg_mean_squared_error",
                                   cv=3,
                                   verbose=1,
                                   population_size=10,
                                   gene_mutation_prob=0.10,
                                   gene_crossover_prob=0.5,
                                   tournament_size=3,
                                   generations_number=5,
                                   n_jobs=1)
ga1.fit(valid_X, valid_y)
print(ga1.best_params_)
print("MSE:"+ str(-ga1.best_score_))

Types [1, 1, 1, 1, 1, 2] and maxint [2, 2, 129, 79, 1, 1] detected
--- Evolve in 374400 possible combinations ---


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  if __name__ == '__main__':


gen	nevals	avg         	min       	max         	std       
0  	10    	-0.000655119	-0.0026869	-0.000378133	0.00067851
1  	9     	-0.000406454	-0.000461557	-0.000378133	3.15318e-05
2  	3     	-0.000382603	-0.000401389	-0.000378133	8.94984e-06
3  	5     	-0.000378133	-0.000378133	-0.000378133	0          
4  	4     	-0.000378133	-0.000378133	-0.000378133	0          
5  	6     	-0.00038465 	-0.00044331 	-0.000378133	1.95531e-05
Best individual is: {'activation': 'relu', 'batch_size': 250, 'units': 104, 'units_1': 95, 'epochs': 1000, 'lr': 0.0005}
with fitness: -0.0003781325900783341
{'activation': 'relu', 'batch_size': 250, 'units': 104, 'units_1': 95, 'epochs': 1000, 'lr': 0.0005}
MSE:0.0003781325900783341


In [7]:
save = pd.DataFrame(ga1.cv_results_).sort_values("mean_test_score", ascending=False)
save.to_csv("stock_dataset/hasil_hyper_bca.csv", index=False)
save

,param_index,index,params,mean_test_score,std_test_score,min_test_score,max_test_score,nan_test_score?
0,0,11,"{'activation': 'relu', 'batch_size': 450, 'uni...",-0.000378,0.0,-0.000378,-0.000378,False
14,0,28,"{'activation': 'relu', 'batch_size': 250, 'uni...",-0.000378,0.0,-0.000378,-0.000378,False
25,0,40,"{'activation': 'relu', 'batch_size': 250, 'uni...",-0.000378,0.0,-0.000378,-0.000378,False
24,0,39,"{'activation': 'relu', 'batch_size': 250, 'uni...",-0.000378,0.0,-0.000378,-0.000378,False
23,0,38,"{'activation': 'relu', 'batch_size': 250, 'uni...",-0.000378,0.0,-0.000378,-0.000378,False
22,0,37,"{'activation': 'relu', 'batch_size': 250, 'uni...",-0.000378,0.0,-0.000378,-0.000378,False
21,0,36,"{'activation': 'relu', 'batch_size': 250, 'uni...",-0.000378,0.0,-0.000378,-0.000378,False
20,0,35,"{'activation': 'relu', 'batch_size': 250, 'uni...",-0.000378,0.0,-0.000378,-0.000378,False
19,0,34,"{'activation': 'relu', 'batch_size': 250, 'uni...",-0.000378,0.0,-0.000378,-0.000378,False
18,0,33,"{'activation': 'relu', 'batch_size': 250, 'uni...",-0.000378,0.0,-0.000378,-0.000378,False
